**Web-scraped analysis of potential whitespace on garmentory.com.**

Sell through estimated by (all sizes - sizes available).

Download & open plotly HTML to view full-screen chart ('files', top left)

Summmary statistics by category and brand in .csv format as well.

In [ ]:
import pandas as pd
df = pd.read_csv('/content/garmentory - Sheet2.csv')
df['Percent Sell Through'] = (df['total sizes']-df['num_available'])/df['total sizes']

df_ = df[df['price_range']!='$0-$75 Price Point']
group_columns = ['Base Category', 'Sub Category', 'price_range']
brand_columns = ['Base Category', 'Sub Category', 'price_range','Brand']
summary_columns = ['Percent Discount', 'Percent Sell Through']

summary_stats = df_.groupby(group_columns)[summary_columns].agg(['mean', 'min', 'max'])
summary_stats_sorted = summary_stats.sort_values(('Percent Sell Through', 'mean'), ascending=False)
summary_stats_sorted.to_csv('stats_category.csv')

brand_stats = df_.groupby(brand_columns)[summary_columns].agg(['mean', 'min', 'max'])
brand_stats_sorted = summary_stats.sort_values(('Percent Sell Through', 'mean'), ascending=False)
brand_stats_sorted.to_csv('stats_brand.csv')

import plotly.express as px
import pandas as pd
df['percent_discount'] = df['Percent Discount'].apply(lambda x: round(x, 1))
fig3 = px.treemap(df, path=['Base Category', 'Sub Category','price_range', 'Brand' ,'percent_discount'], values='counter', color='Percent Sell Through')
fig3.show()
fig3.write_html("download_and_click.html")

In [ ]:
#website scraping code

In [ ]:
import pandas as pd
import numpy as np
import requests
import time
import bs4
from bs4 import BeautifulSoup
import random
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36'}

In [ ]:
dff = pd.read_csv('/content/garmentory - Sheet1 (1).csv')
url = 'https://www.garmentory.com/sale/all/women-flats-mules?new=1'
df = {'name':[],'brand':[],'category':[],'sub_category':[],'link':[],'price':[],'discounted_price':[],'sizes':[],'store':[]}

for ii in range(len(dff)):
  for iii in range(10):
    url = dff['Link'][ii]+'?page='+str(iii+1)
    category = dff['Cat'][ii]
    sub_category = dff['Sub Cat'][ii]
    html = requests.get(url,headers = headers).text
    google_soup = str(bs4.BeautifulSoup(html, "html.parser"))
    google_soup = google_soup.split('&quot;,&quot;name&quot;:&quot;')[7:]
    if len(google_soup)==1:
      break
    for i in range(len(google_soup)-1):
      try:

        name = google_soup[i+1].split(';,')[0]
        brand = google_soup[i+1].split(';brand&quot;:&quot;')[1].split('&quot')[0]
        link = google_soup[i+1].split(';,&quot;path&quot;:&quot;')[1].split('&quot')[0]
        price = google_soup[i+1].split('&quot;retailPrice&quot;:&quot;$')[1].split('&quot')[0].replace('.00','')
        discounted_price = google_soup[i+1].split('price&quot;:&quot;$')[1].split('&quot')[0].replace('.00','')
        sizes = google_soup[i+1].split('sizes&quot;:&quot;')[1].split('&quot')[0]
        stores = google_soup[i+1].split('&quot;store&quot;:&quot;')[1].split('&quot')[0]

        df['name'].append(name)
        df['brand'].append(brand)
        df['link'].append(link)
        df['price'].append(price)
        df['discounted_price'].append(discounted_price)
        df['sizes'].append(sizes)
        df['store'].append(stores)
        df['category'].append(category)
        df['sub_category'].append(sub_category)

      except:
        print('ERROR')

df = pd.DataFrame(df).drop_duplicates(subset = 'link').reset_index().drop(columns = 'index')

all_sizes = []
for ggs in range(len(df)):
  url = 'https://www.garmentory.com'+df['link'][ggs]
  html = requests.get(url,headers = headers).text
  google_soup = str(bs4.BeautifulSoup(html, "html.parser"))
  lenn = str(len(google_soup.split('><label for="size'))-1)
  all_sizes.append(lenn)

df['total sizes'] = all_sizes

df.to_csv('saved_garmentory.csv')

df.head(3)

ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR


In [ ]:
df = pd.read_csv('/content/garmentory - Sheet1 (2).csv')
num_available = []
for i in range(len(df)):
  num_available.append(len(df['sizes'][i].split(',')))
df['num_available'] = num_available
df['sell_through'] = df['total sizes']-df['num_available']
p_discount = []
for i in range(len(df)):
  if df['p_discount'][i]<=0:
    p_discount.append(.0001)
  else:
    p_discount.append(df['p_discount'][i])
sell_through = []
for i in range(len(df)):
  if df['sell_through'][i]<=0:
    sell_through.append(.0001)
  else:
    sell_through.append(df['sell_through'][i])

pp = []
pr = []
for i in range(len(df['price'])):
  pr.append(df['price'][i].replace(',',''))

df['price'] = pr
df['price'] = df['price'].replace(',','').astype(float)
for i in range(len(df['price'])):
  if df['price'][i]>=350:
    pp.append('$350+ Price Point')
  elif (df['price'][i]>=250)&(df['price'][i]<=350):
    pp.append('$250-$350 Price Point')
  elif (df['price'][i]>=150)&(df['price'][i]<=250):
    pp.append('$150-$250 Price Point')
  elif (df['price'][i]>=75)&(df['price'][i]<=150):
    pp.append('$75-$150 Price Point')
  elif (df['price'][i]>=0)&(df['price'][i]<=75):
    pp.append('$0-$75 Price Point')

df['price_range'] = pp


df['sell_through']=sell_through
df['p_discount']=p_discount
df['counter'] = 1
df.to_csv('saved_final_column_rename.csv')

In [ ]:
import plotly.express as px
fig3 = px.treemap(df, path=['Base Category', 'Sub Category','price_range', 'Brand'], values='counter', color='Percent Discount')
fig3.show()
fig3.write_html("product_count_discountColor.html")